# **Data Acquisition**
 # Instructor: K.M.Asif
  # Author: M.Haris Iqbal 



# **Hit of Covid-19 and its impact on different economies**

### This project aims to identify the hit of Covid-19 and its impact on different economies. This involves comparing the GDP of different economies before and after the Covid-19. Various data sources are used to determine the behavior of different economies of the world due to the Covid-19. 

# Data sources used in the project:



1.   https://www.worldometers.info/coronavirus/ : contains the cumulative cases of Covid-19 (worlwide) in a tabular form. 
2.   https://raw.githubusercontent.com/hicala/gdp_python-data-mining/main/Per%20the%20World%20Bank%20(2019).csv : contains the GDP (US$million) of 2019 (worldwide) in a CSV format.
3. https://raw.githubusercontent.com/adamkissinger/world-gdp-change/main/gdp_growth.csv: contanis the GDP growth (worldwide) from 1960-2020 in a CSV format.





**FYI: After running all the cells for the first time, please disconnect and delete runtime in case you want to run it again.**

Importing the necessary libraries.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 
import sqlite3
import io
import csv

We created a function `format_column_names(column_names)` which takes in `column_names` as a parameter and returns the updated column names which have no special characters such as `" ", /, ()`. 

In [2]:
def format_column_names(column_names):
  col_names = []
  for column_name in column_names:
    if column_name.isdigit() or " " in column_name or "/" in column_name or "(" in column_name or ")" in column_name:
      col_names.append(f'"{column_name}"')
    else:
      col_names.append(column_name)
  return col_names

We implemented a function `create_sql_table` which creates a **SQL table** in a database with the given parameters: `database_name`, `table_name`, `formatted_column_names`, `data_types`. The `sqlite3.connect()` creates a connection to the database and `conn.cursor()` creates a cursor to execute SQL commands.

We use the `zip` function to to pair each column name with its corresponding data type from the `formatted_column_names` and `data_types` lists, respectively. We then use a `for-loop` to iterate over these pairs and build the **CREATE TABLE** statement. 

In the line **query += query[:-2] + ")"** the [:-2] part is used to remove the last two characters of the query string, which are **", "** (a comma and a space). We then concatenate the closing parenthesis **")"**  to the end of the query string, resulting in a **CREATE TABLE** statement that has the correct syntax. Finally, the query is executed using the `cursor.execute()` method. This will create the table in the database with the given column names and data types.

In [3]:
def create_sql_table(database_name, table_name, formatted_column_names, data_types):

  conn = sqlite3.connect(database_name)
  cursor = conn.cursor()

  query = "CREATE TABLE " + table_name + "("
  for column_name, data_type in zip(formatted_column_names, data_types):
      query += column_name + " " + data_type + ", "
  query = query[:-2] + ")" 
  cursor.execute(query) 

We created another function `sql_insert_many` which takes in the parameters: `db`, `qry`, `data`. We used `executemany()` which iterates through the sequence of parameters, each time passing the current parameters to the `execute()` method. The `commit()` command is used for storing changes performed by a transaction.

In [4]:
def sql_insert_many(db, qry, data):
  conn = sqlite3.connect(db)
  cur = conn.cursor()
  cur.executemany(qry, data)
  conn.commit()

We used the first `url` of the data source and then assigned it to a variable. Then, we used the `requests` library and the `get` method (function) to get the meta data of the url.

In [5]:
url = 'https://www.worldometers.info/coronavirus/'
page = requests.get(url)

We used `BeautifulSoup` library to use the html source code  extracted above to create a `lxml` parse tree. There are three tables in our parse tree, but we just needed the very first table containing today's Covid-19 data so we specified its `id` to get it. 

In [6]:
soup = BeautifulSoup(page.text, 'lxml')
table1 = soup.find('table', id='main_table_countries_today')

We created an empty list and used the `for-loop` to iterate through all of the data using the `find_all`, which finds all the table headers (`th`). After that we used `.text` to get the required data inside the `th` tag. Then we append all the headers into our list. 

In [7]:
header_row = []
for i in table1.find_all('th'):
 title = i.text
 header_row.append(title)

We renamed some of the `header_row` (columns) as needed. 

In [8]:
header_row[0] = 'num'
header_row[1] = 'country'
header_row[9] = 'critical'
header_row[13] = 'Tests per 1M pop'

We created an empty list and then used the `for-loop` to iterate through all of the `tr` tags ignoring the index. By using `find_all` we find all the `td` tags from the data. Then using list comprehension, we used `.text` on all of the `td` tags to create list of lists (LoL) of all the countries data. After that we used the `append()` to assign the individual rows to the empty list created earlier to create a LoL. All the rows are located under tag `tr` and all the data items are located under the tag `td`.

In [9]:
row_lol = []

for j in table1.find_all('tr')[1:]:
 row_data = j.find_all('td')

 row = [i.text for i in row_data]
 row_lol.append(row)

In the table, the first seven rows displayed are the continent data. Since this is of no use to us, we skip the first 7 rows of the LoL. Then, we gather rest of the data.

In [10]:
row_lol = row_lol[7:]
row_lol = row_lol[0:232]

We used a nested `for-loop` where the outer loop iterates through the LoL created above and the inner loop iterates through the strings in each row of the LoL. Then, we used `rstrip()` to remove trailing spaces from the each string in the LoL.

In [11]:
for row in row_lol:
    for i in range(len(row)):
        row[i] = row[i].rstrip()

We named our database as **covid.db** and created our first table. First we create a connection to our database using `sqlite3.connect()` and then create a cursor to execute SQL commands using `conn.cursor()`. After that we assigned the `data_types` of our `header_row` (columns) as needed and then named our table. After that we called the `create_sql_table()` function from above which takes in the specified parameters and eventually creates the table. 

We created a placeholder string for insert query depending on the length of `header_row`. To achieve this, we used `join()` function after getting a **", "** (comma and a space) in our string which takes all items in an iterable and joins them into one string. Inside the `join()`, we used a list comprehension which iterates through the length of the `header_row` and **"IFNULL(?, NULL)"** replaces any empty string with NULL. After that we created an insert query based on the number of `header_row` and then called the `sql_insert_many()` function to populate the table.

In [12]:
db = 'covid.db'

conn = sqlite3.connect(db)
cursor = conn.cursor()

data_types = ["INTEGER" for _ in range(len(header_row))]
data_types[1] = 'VARCHAR(100)'

table_name = "covid_data"
create_sql_table(db, table_name, format_column_names(header_row), data_types)

placeholder = ", ".join(["IFNULL(?, NULL)" for _ in range(len(header_row))]) 

iqry = f"INSERT INTO {table_name} VALUES({placeholder})" 

sql_insert_many(db, iqry, row_lol)

The scraping for the first data source is now complete and its table has been created and populated in the database.

Now, we begin processing data of our second data source which is a csv and has been accessed from github. We assign the variable `url` to the link of the data source. After that with the use of `requests` and `get` we get the raw data accessed from the url and assign to variable `s`. 

In [13]:
url = "https://raw.githubusercontent.com/hicala/gdp_python-data-mining/main/Per%20the%20World%20Bank%20(2019).csv"
s = requests.get(url).content 

Earlier, we imported `io` which decodes diffrent kind of I/Os to give us readable data from the raw file.  Here, we used `io` to decode the raw data from the format passed and assigned it to the variable `data`. Then we used `seek()` to go to the start of the decoded string and from there, we used `readlines()` to read all the lines in the decoded continuous string separated into lines by `\n`. After that we selected the first continuous string which will be our columns and assigned it to the variable `header_row`. Initially the string is in raw form, so we used `strip()` to remove any leading or trailing whitespace characters from the string. Then we also used the `split()` on the resulting string, which splits the string on the occurance of each comma ( , ) to create a list of items. 

After that we selected all the rest of the continuous strings and assigned those to the variable `data_rows`. Then we used `csv.reader()` on `data_rows` which reads each line separated by `\n` in the continuous string and splits the string on the occurance of each delimiter and then returns a reader object which will iterate over lines in the `data_rows`. Then we created an empty list and used a `for-loop` to iterate through each row from all the lines and append the row data to the list to create a LoL.

In [14]:
data = io.StringIO(s.decode("ISO-8859-1"))

data.seek(0)
lines = data.readlines() 

header_row = lines[0] 
header_row = header_row.strip().split(',') 

data_rows = lines[1:]
reader = csv.reader(data_rows)

row_lol = []
for row in reader: 
    row_lol.append(row)

After that we assigned the `data_types` of our `header_row` (columns) as needed and then named our table. After that we called the `create_sql_table()` function from above which takes in the specified parameters and eventually creates the table. 

We created a placeholder string for insert query depending on the length of `header_row`. To achieve this, we used `join()` function after getting a **", "** (comma and a space) in our string which takes all items in an iterable and joins them into one string. Inside the `join()`, we used a list comprehension which iterates through the length of the `header_row` and **"IFNULL(?, NULL)"** replaces any empty string with NULL. After that we created an insert query based on the number of `header_row` and then called the `sql_insert_many()` function to populate the table.

In [15]:
data_types = ['VARCHAR(100)', 'INTEGER', 'INTEGER']

table_name = "gdp2019"
create_sql_table(db, table_name, format_column_names(header_row), data_types)

placeholder = ", ".join(["IFNULL(?, NULL)" for _ in range(len(header_row))]) 

iqry = f"INSERT INTO {table_name} VALUES({placeholder})" 

sql_insert_many(db, iqry, row_lol)

The second data source has been processed and its table has been created and populated in the database.

Now, we move onto the third data source  which is also a csv and has been accessed from github. We assign the variable `url` to the link of the data source. After that with the use of `requests` and `get` we get the raw data accessed from the url and assign to variable `s`. 

In [16]:
url = "https://raw.githubusercontent.com/adamkissinger/world-gdp-change/main/gdp_growth.csv"
s = requests.get(url).content

Most of the code is identical to the one above. But there's a few differences. This one was encoded in `utf-8`, so we pass that to decode the data and assign it to the variable `data`. Then we used `seek()` to go to the start of the decoded string and from there, we used `readlines()` to read all the lines in the decoded continuous string separated into lines by `\n`. After that we selected the first continuous string which will be our columns and assigned it to the variable `header_row`. Initially the string is in raw form, so we used `strip()` to remove any leading or trailing whitespace characters from the string. Then we also used the `split()` on the resulting string, which splits the string on the occurance of each comma ( , ) to create a list of items. In our `header_row` there was a empty column `Unnamed: 65`, so we sliced it out of our list.

After that we selected all the rest of the continuous strings and assigned those to the variable `data_rows`. Then we used `csv.reader()` on `data_rows` which reads each line separated by `\n` in the continuous string and splits the string on the occurance of each delimiter and then returns a reader object which will iterate over lines in the `data_rows`. Then we created an empty list and used a `for-loop` to iterate through each row from all the lines and append the row data to the list to create a LoL.

In [17]:
data = io.StringIO(s.decode("utf-8"))

data.seek(0)
lines = data.readlines() 

header_row = lines[0]
header_row = header_row.strip().split(',')
header_row = header_row[0: -1]

data_rows = lines[1:]
reader = csv.reader(data_rows)

row_lol = []
for row in reader: 
    row_lol.append(row[0: -1])

After that we assigned the `data_types` of our `header_row` (columns) as needed and then named our table. After that we called the `create_sql_table()` function from above which takes in the specified parameters and eventually creates the table. 

We created a placeholder string for insert query depending on the length of `header_row`. To achieve this, we used `join()` function after getting a **", "** (comma and a space) in our string which takes all items in an iterable and joins them into one string. Inside the `join()`, we used a list comprehension which iterates through the length of the `header_row` and **"IFNULL(?, NULL)"** replaces any empty string with NULL. After that we created an insert query based on the number of `header_row` and then called the `sql_insert_many()` function to populate the table.

In [18]:
data_types = ["INTEGER" for _ in range(len(header_row))] 

data_types[0] = "VARCHAR(100)" 
data_types[1] = "VARCHAR(100)" 

table_name = "gdpall"
create_sql_table(db, table_name, format_column_names(header_row), data_types)

placeholder = ", ".join(["IFNULL(?, NULL)" for _ in range(len(header_row))]) 

iqry = f"INSERT INTO {table_name} VALUES({placeholder})" 

sql_insert_many(db, iqry, row_lol)

The third data source has been processed and its table has been created and populated in the database.

Now, for the use of these tables to answer our central question, we plan to use the first table to compare the tangible impact of covid-19 on different countries. The second table shows us the economy of those countries right before the hit of covid. The third table explains the change in gdp through a span of years, so we can estimate the trajectory economy was heading in before covid and what actually transpired when covid-19 hit. We'll merge these using the country names and with the help of plotly, visually represent the impact covid-19 had on the economic conditions of a country. 